Status of Notebook: Work in Progress

In [1]:
import torch
import random
import torch
import torch.nn as nn
import math
import time
import numpy as np

### Download the Data

In [6]:
# uncomment to download the datasets
#!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/ptb/test.txt
#!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/ptb/train.txt
#!wget https://raw.githubusercontent.com/neubig/nn4nlp-code/master/data/ptb/valid.txt

### Process the Data

In [2]:
# function to read in data, process each line and split columns by " ||| "
def read_data(filename):
    data = []
    with open(filename, "r") as f:
        for line in f:
            line = line.strip().split(" ")
            data.append(line)
    return data

# read the data
train_data = read_data('data/ptb/train.txt')
val_data = read_data('data/ptb/valid.txt')

# creating the word and tag indices and special tokens
word_to_index = {}
index_to_word = {}
word_to_index["<s>"] = len(word_to_index)
index_to_word[len(word_to_index)-1] = "<s>"
word_to_index["<unk>"] = len(word_to_index) # add <UNK> to dictionary
index_to_word[len(word_to_index)-1] = "<unk>"

# create word to index dictionary and tag to index dictionary from data
def create_dict(data, check_unk=False):
    for line in data:
        for word in line:
            if check_unk == False:
                if word not in word_to_index:
                    word_to_index[word] = len(word_to_index)
                    index_to_word[len(word_to_index)-1] = word
            
            # has no effect because data already comes with <unk>
            # should work with data without <unk> already processed
            else: 
                if word not in word_to_index:
                    word_to_index[word] = word_to_index["<unk>"]
                    index_to_word[len(word_to_index)-1] = word

create_dict(train_data)
create_dict(val_data, check_unk=True)

# create word and tag tensors from data
def create_tensor(data):
    for line in data:
        yield([word_to_index[word] for word in line])

train_data = list(create_tensor(train_data))
val_data = list(create_tensor(val_data))

number_of_words = len(word_to_index)

In our implementation we are using batched training. There are a few differences from the original implementation found [here](https://github.com/neubig/nn4nlp-code/blob/master/02-lm/loglin-lm.py). 

### Define the Model

In [3]:
## define the model

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# length of the n-gram
N = 2

# logilinear model
class LogLinear(nn.Module):
    def __init__(self, number_of_words, ngram_length):
        super(LogLinear, self).__init__()

        # different lookups for each position in the n-gram
        self.embeddings = nn.ModuleList([nn.Embedding(number_of_words, number_of_words) for _ in range(ngram_length)])
        self.bias = torch.zeros(number_of_words, requires_grad=True).type(torch.FloatTensor).to(device)

        # initialize
        for i in range(N):
            nn.init.xavier_uniform_(self.embeddings[i].weight)

    def forward(self, x):
        # calculate score
        embs = torch.cat([lookup(x) for x, lookup in zip(x.T, self.embeddings)]).view(N, x.shape[0], -1) # N x batch_size x embedding_size
        embs = torch.sum(embs, dim=0) # batch_size x embedding_size
        scores = embs + self.bias
        
        return scores

### Model Settings and Functions

In [10]:
model = LogLinear(number_of_words, N)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.to(device)

# function to calculate the sentence loss
def calc_sent_loss(sent):
    S = word_to_index["<s>"]
    
    # initial history is equal to end of sentence symbols
    hist = [S] * N
    
    # collect all target and histories
    all_targets = []
    all_histories = []
    
    # step through the sentence, including the end of sentence token
    for next_word in sent + [S]:
        all_histories.append(list(hist))
        all_targets.append(next_word)
        hist = hist[1:] + [next_word]

    logits = model(torch.LongTensor(all_histories).to(device))
    loss = criterion(logits, torch.LongTensor(all_targets).to(device))

    return loss

MAX_LEN = 100
# Function to generate a sentence
def generate_sent():
    S = word_to_index["<s>"]
    hist = [S] * N
    sent = []
    while True:
        logits = model(torch.LongTensor([hist]).to(device))
        p = torch.nn.functional.softmax(logits) # 1 x number_of_words
        next_word = p.multinomial(num_samples=1).item()
        if next_word == S or len(sent) == MAX_LEN:
            break
        sent.append(next_word)
        hist = hist[1:] + [next_word]
    return sent

### Train the Model

In [11]:
# start training
for ITER in range (10): # CHANGE to 100
    # training
    random.shuffle(train_data)

    model.train()
    train_words, train_loss  = 0, 0.0
    for sent_id, sent in enumerate(train_data[1:1000]): # CHANGE to all train_data
        
        my_loss = calc_sent_loss(sent)
        
        train_loss += my_loss.item()
        train_words += len(sent)

        optimizer.zero_grad()
        my_loss.backward()
        optimizer.step()

        if (sent_id+1) % 5000 == 0:
            print("--finished %r sentences" % (sent_id+1))
    print("iter %r: train loss/word=%.4f, ppl=%.4f" % (ITER, train_loss/train_words, math.exp(train_loss/train_words)))

    # evaluation
    model.eval()
    dev_words, dev_loss = 0, 0.0
    start = time.time()
    for sent_id, sent in enumerate(val_data):
        my_loss = calc_sent_loss(sent)
        dev_loss += my_loss.item()
        dev_words += len(sent)
    print("iter %r: dev loss/word=%.4f, ppl=%.4f, time=%.2fs" % (ITER, dev_loss/dev_words, math.exp(dev_loss/dev_words), time.time()-start))

    # Generate a few sentences
    for _ in range(5):
        sent = generate_sent()
        print(" ".join([index_to_word[x] for x in sent]))

iter 0: train loss/word=0.3799, ppl=1.4621
iter 0: dev loss/word=0.3860, ppl=1.4710, time=1.20s
the dollar and it was n't the only at the national <unk> last to the <unk> <unk>
i think the importance of <unk>
the dollar began friday on a new
the purchase of the transaction
but even mr. boren added combination wall street firms developed clarify judgment roads current joel announce services enthusiasts jeffrey trades nor quite school highlight co-chief manpower unveil frustration plunged admits investigator spent sdi museum exchanged passenger interpublic interbank prosecutorial undo earn base relevant ounces cray-3 cellular harmful ultimate wells co


/opt/conda/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


iter 1: train loss/word=0.3836, ppl=1.4676
iter 1: dev loss/word=0.3949, ppl=1.4842, time=1.18s
the <unk> which has been a recently acquired british air is said a big stance major market activity
i dominates so slightly in the current N N
consumer product sales global viewpoint wastewater sensitive sweetened professionals mather are all of them and <unk> to the move quickly unprecedented maturing in N as N in cash and <unk> that 's not
an incident in N to $ N
an incident in N the increase in and <unk> just in panama
iter 2: train loss/word=0.3819, ppl=1.4650
iter 2: dev loss/word=0.4004, ppl=1.4924, time=1.16s
perhaps he is n't a federal judge <unk> the end of the natural will probably try to gain from its traditional <unk> new york stock exchange composite trading
u.s. wants to buy her sell to the indictment
he is a <unk> so far has been painful for renaissance
am radio which has been dogged in beijing
it 's an <unk> in u.s.-soviet affairs
iter 3: train loss/word=0.3743, ppl=1.4539
it